# Mục tiêu
Notebook này thực hiện quy trình làm sạch và chuẩn bị dữ liệu (Data Preprocessing) cho tập dữ liệu quan trắc không khí. Mục tiêu là chuyển đổi dữ liệu thô thành dạng chuẩn, sẵn sàng cho việc phân tích hoặc huấn luyện mô hình.

**Các bước thực hiện chính:**
- **Xử lý dữ liệu không hợp lệ (Invalid Data)**
- **Xử lý giá trị thiếu (Missing Values)**
- **Xử lý trùng lặp (Duplicates)** 
- **Phân tích giá trị ngoại lai (Outliers)** 

### Import các thư viện cần thiết

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser
import sys, importlib
sys.path.insert(0, os.path.abspath(".."))
import src.preprocess as preprocess
importlib.reload(preprocess)
from src.preprocess import (check_missing, count_outliers_iqr, aqi_to_status)

### Đọc dữ liệu từ file csv

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

PROJECT_ROOT = os.path.abspath(os.path.join(".."))
RAW_DATA_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "air_quality.csv")
df = pd.read_csv(RAW_DATA_PATH)
print("Data shape:", df.shape)
df.head()

/tmp/ipykernel_984/1779837074.py:6: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_PATH)


Data shape: (5882208, 25)


,date,sitename,county,aqi,pollutant,status,so2,co,o3,o3_8hr,pm10,pm2.5,no2,nox,no,windspeed,winddirec,unit,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid
0,2024-08-31 23:00,Hukou,Hsinchu County,62.0,PM2.5,Moderate,0.9,0.17,35.0,40.2,18.0,17.0,2.3,2.6,0.3,2.3,225,NaN,0.2,20.1,26.0,1.0,121.038869,24.900097,22.0
1,2024-08-31 23:00,Zhongming,Taichung City,50.0,NaN,Good,1.6,0.32,27.9,35.1,27.0,14.0,7.6,9.3,1.6,1.1,184,NaN,0.2,15.3,23.0,1.0,120.641092,24.151958,31.0
2,2024-08-31 23:00,Zhudong,Hsinchu County,45.0,NaN,Good,0.4,0.17,25.1,40.6,21.0,13.0,2.9,4.1,1.1,0.4,210,NaN,0.2,13.8,24.0,0.0,121.088955,24.740914,23.0
3,2024-08-31 23:00,Hsinchu,Hsinchu City,42.0,NaN,Good,0.8,0.2,30.0,35.9,19.0,10.0,4.0,4.8,0.7,1.9,239,NaN,0.2,13.0,26.0,1.0,120.972368,24.805636,24.0
4,2024-08-31 23:00,Toufen,Miaoli County,50.0,NaN,Good,1.0,0.16,33.5,35.9,18.0,14.0,1.8,3.1,1.2,1.8,259,NaN,0.1,15.3,28.0,1.0,120.898693,24.696907,25.0


### Xem xét kiểu dữ liệu của các cột

In [3]:
df.dtypes

date          object
sitename      object
county        object
aqi          float64
pollutant     object
status        object
so2           object
co            object
o3            object
o3_8hr        object
pm10          object
pm2.5         object
no2           object
nox           object
no            object
windspeed     object
winddirec     object
unit         float64
co_8hr        object
pm2.5_avg     object
pm10_avg      object
so2_avg       object
longitude    float64
latitude     float64
siteid       float64
dtype: object

### Nhận xét

- Nhiều cột numeric (so2, co, pm2.5, windspeed, winddirec…) đang là object → cần chuyển sang numeric.

- date nên chuyển sang datetime.

In [4]:
# Tạo bản sao để xử lý
df_cleaned = df.copy()

## I. Xử lý các giá trị không hợp lệ

### 1. Kiểm tra numeric columns
#### a. Các cột numeric ("so2", "co",…) phải ≥ 0.

In [5]:
numeric_cols = ["so2", "co", "o3", "o3_8hr", "pm10", "pm2.5",
                "no2", "nox", "no", "co_8hr",
                "pm2.5_avg", "pm10_avg", "so2_avg",
                "windspeed","aqi", "winddirec"]

# Chuyển sang numeric, lỗi → NaN
for col in numeric_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

In [6]:
# Kiểm tra giá trị âm
for col in numeric_cols:
    invalid = df_cleaned[df_cleaned[col] < 0]
    if not invalid.empty:
        print(f"Column {col} có {len(invalid)} giá trị âm.")

Column so2 có 8028 giá trị âm.
Column co có 412 giá trị âm.
Column o3 có 478 giá trị âm.
Column o3_8hr có 50 giá trị âm.
Column pm10 có 2 giá trị âm.
Column pm2.5 có 3 giá trị âm.
Column no2 có 1029 giá trị âm.
Column nox có 160 giá trị âm.
Column no có 29470 giá trị âm.
Column co_8hr có 35 giá trị âm.
Column pm2.5_avg có 6 giá trị âm.
Column pm10_avg có 6 giá trị âm.
Column so2_avg có 12 giá trị âm.
Column windspeed có 72 giá trị âm.
Column aqi có 7391 giá trị âm.


In [7]:
# Loại các giá trị âm (pollutant, windspeed, aqi)
for col in numeric_cols:
    df_cleaned.loc[df_cleaned[col] < 0, col] = pd.NA

In [8]:
# Kiểm tra giá trị âm
for col in numeric_cols:
    invalid = df_cleaned[df_cleaned[col] < 0]
    if not invalid.empty:
        print(f"Column {col} có {len(invalid)} giá trị âm.")

#### Đã loại bỏ các giá trị âm

#### b. Kiểm tra aqi

- Miền giá trị logic của `aqi` là **0 – 500**.  
- Giá trị <= 0 hoặc > 500 được coi là không hợp lệ và sẽ được chuyển thành `NaN` để xử lý missing value.

In [9]:
df_cleaned.loc[df_cleaned["aqi"] < 0, "aqi"] = pd.NA
df_cleaned["aqi_over_500"] = df_cleaned["aqi"] > 500

print("Số AQI > 500:", df_cleaned["aqi_over_500"].sum())

Số AQI > 500: 0


In [10]:
# Xóa cột tạm đánh dấu
df_cleaned.drop(columns=["aqi_over_500"], inplace=True)

#### c. Kiểm tra "winddirec"

- Miền giá trị hợp lệ: **0 – 360 độ**.  
- Giá trị < 0 hoặc > 360 được coi là không hợp lệ và sẽ bị xóa.  

In [11]:
n_invalid_wd = (
    (df_cleaned["winddirec"] < 0) |
    (df_cleaned["winddirec"] > 360)
).sum()

if n_invalid_wd > 0:
    print(f"winddirec: {n_invalid_wd} giá trị ngoài [0, 360]")
    df_cleaned.loc[
        (df_cleaned["winddirec"] < 0) |
        (df_cleaned["winddirec"] > 360),
        "winddirec"
    ] = pd.NA

winddirec: 578 giá trị ngoài [0, 360]


#### d. Kiểm tra "siteid"

- `siteid` là **ID duy nhất của mỗi trạm quan trắc**.  
- Giá trị phải là **số nguyên dương** và tương ứng với dữ liệu thực tế của từng trạm.  
- Nếu dataset có `siteid = 0` hoặc giá trị không hợp lệ → cần xử lý:
  - Đổi 0 hoặc giá trị không hợp lệ thành `NaN`.  

In [12]:
# 1. Kiểm tra siteid <= 0
invalid_siteid = (df_cleaned["siteid"] <= 0).sum()
print(f"Số dòng có siteid <= 0: {invalid_siteid}")

# 2. Đổi các giá trị không hợp lệ thành NaN
df_cleaned.loc[df_cleaned["siteid"] <= 0, "siteid"] = pd.NA


Số dòng có siteid <= 0: 28


### 2. Kiểm tra date
- Chuyển cột `date` sang kiểu `datetime`.  
- Các giá trị lỗi định dạng sẽ thành `NaT` và được loại bỏ.  
- Đảm bảo dữ liệu thời gian hợp lệ để sắp xếp, lọc và phân tích chuỗi thời gian.

In [13]:
# df_cleaned["date"] có 2 format:
#   1. "YYYY-MM-DD HH:MM"
#   2. "YYYY/MM/DD HH:MM:SS"

# Thử parse format đầu tiên ("YYYY-MM-DD HH:MM")
date_minute = pd.to_datetime(
    df_cleaned["date"], 
    format="%Y-%m-%d %H:%M", 
    errors='coerce'
)

# Với những NaT, thử parse format thứ hai ("YYYY/MM/DD HH:MM:SS")
mask = date_minute.isna()
date_second = pd.to_datetime(
    df_cleaned.loc[mask, "date"], 
    format="%Y/%m/%d %H:%M:%S", 
    errors='coerce'
)

# Gộp kết quả 
df_cleaned["date"] = date_minute
df_cleaned.loc[mask, "date"] = date_second

# Kiểm tra lỗi parse 
invalid_dates = df_cleaned[df_cleaned["date"].isna()]
print(f"Số dòng date lỗi: {len(invalid_dates)}")
display(invalid_dates)

Số dòng date lỗi: 0


,date,sitename,county,aqi,pollutant,status,so2,co,o3,o3_8hr,pm10,pm2.5,no2,nox,no,windspeed,winddirec,unit,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid


### Khám phá dữ liệu sau khi thực hiện chuyển đổi

In [14]:
print(df_cleaned["date"])

0         2024-08-31 23:00:00
1         2024-08-31 23:00:00
2         2024-08-31 23:00:00
3         2024-08-31 23:00:00
4         2024-08-31 23:00:00
                  ...        
5882203   2016-11-25 13:00:00
5882204   2016-11-25 13:00:00
5882205   2016-11-25 13:00:00
5882206   2016-11-25 13:00:00
5882207   2016-11-25 13:00:00
Name: date, Length: 5882208, dtype: datetime64[ns]


## II. Xử lý missing value

Dữ liệu quan trắc thường bị thiếu do lỗi cảm biến hoặc đường truyền. Chúng ta áp dụng chiến lược xử lý theo từng loại biến:


In [15]:
missing_info = check_missing(df)
display(missing_info)

Total rows: 5,882,208
Columns with missing values: 22 / 25



,missing_count,missing_percent (%)
unit,5882208,100.00
pollutant,3235012,55.00
siteid,1779469,30.25
latitude,933620,15.87
longitude,933620,15.87
so2_avg,629162,10.70
winddirec,219736,3.74
windspeed,219498,3.73
nox,169124,2.88
no,169455,2.88


### 1. Loại bỏ dữ liệu không cần thiết
* **Cột `unit`**: Thiếu 100% dữ liệu $\rightarrow$ **Xóa**.
* **Cột `pollutant`**: Thiếu 55% và thông tin $\rightarrow$ **Xóa**.


In [16]:
# Xóa các cột không cần thiết
cols_to_drop = ["unit", "pollutant"]
df_cleaned = df_cleaned.drop(columns=[c for c in cols_to_drop if c in df_cleaned.columns])


### 2. Khôi phục thông tin định danh (Site Info)
* **`siteid`**: Được khôi phục dựa trên cặp (`sitename`, `county`). Nếu không thể khôi phục, dòng đó sẽ bị xóa vì không xác định được nguồn gốc.
* **`latitude`, `longitude`**: Được điền (fill) theo `siteid` tương ứng.

#### a. Cột `siteid`

In [17]:
# Khôi phục siteid từ sitename nếu bị missing
df_cleaned["siteid"] = df_cleaned["siteid"].fillna(
    df_cleaned.groupby(["sitename", "county"])["siteid"].transform("first")
)

In [18]:
# 1. Lọc ra các dòng mà siteid vẫn còn là NaN sau khi đã chạy lệnh fillna
rows_failed_to_fill = df_cleaned[df_cleaned["siteid"].isna()]

# 2. In da số lượng tổng
print(f"Tổng số dòng không thể khôi phục siteid: {len(rows_failed_to_fill)}")

# 3. Hiển thị 10 dòng đầu tiên để kiểm tra "mặt mũi" chúng ra sao
print("\n--- 10 dòng ví dụ không điền được ---")
display(rows_failed_to_fill.head(10))

# (Tùy chọn) Xem thử những cái tên trạm (sitename) nào bị lỗi này
print("\n--- Các trạm (sitename) bị lỗi ngày ---")
print(rows_failed_to_fill["sitename"].unique())

Tổng số dòng không thể khôi phục siteid: 1957

--- 10 dòng ví dụ không điền được ---


,date,sitename,county,aqi,status,so2,co,o3,o3_8hr,pm10,pm2.5,no2,nox,no,windspeed,winddirec,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid
1852765,2022-03-04 14:00:00,Zhongming Elementary School,Taichung City,126.0,Unhealthy for Sensitive Groups,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1852845,2022-03-04 13:00:00,Zhongming Elementary School,Taichung City,141.0,Unhealthy for Sensitive Groups,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1852959,2022-03-04 12:00:00,Zhongming Elementary School,Taichung City,152.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1852994,2022-03-04 11:00:00,Zhongming Elementary School,Taichung City,153.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853146,2022-03-04 10:00:00,Zhongming Elementary School,Taichung City,154.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853235,2022-03-04 09:00:00,Zhongming Elementary School,Taichung City,154.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853324,2022-03-04 08:00:00,Zhongming Elementary School,Taichung City,153.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853391,2022-03-04 07:00:00,Zhongming Elementary School,Taichung City,153.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853423,2022-03-04 06:00:00,Zhongming Elementary School,Taichung City,152.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1853533,2022-03-04 05:00:00,Zhongming Elementary School,Taichung City,153.0,Unhealthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN



--- Các trạm (sitename) bị lỗi ngày ---
['Zhongming Elementary School' 'Tainan (Yujing)'
 'Yongkang District Zhongshan South Rd (Tainan University High School)'
 'Mobile Station 03' 'Lulin Mountain' 'Chonglun'
 'Environmental Inspection Station']


Có thể thấy đã bổ sung khá đầy đủ cho các giá trị bị thiếu trong `siteid`
Phần bị thiếu còn lại ta sẽ xóa khỏi dataset

In [19]:
# Xóa các dòng đó
df_cleaned = df_cleaned.dropna(subset=["siteid"])

# Reset index
df_cleaned = df_cleaned.reset_index(drop=True)

# Kiểm tra lại
print("Shape sau khi xóa dòng missing siteid:", df_cleaned.shape)

Shape sau khi xóa dòng missing siteid: (5880251, 23)


#### b. Cột `latitude` và `longtitude`

In [20]:
# Điền missing latitude / longitude theo siteid
df_cleaned["latitude"] = df_cleaned.groupby("siteid")["latitude"].transform(
    lambda x: x.fillna(x.dropna().iloc[0] if not x.dropna().empty else np.nan)
)

df_cleaned["longitude"] = df_cleaned.groupby("siteid")["longitude"].transform(
    lambda x: x.fillna(x.dropna().iloc[0] if not x.dropna().empty else np.nan)
)


In [21]:
# Kiểm tra lại
print("Số dòng còn missing latitude:", df_cleaned["latitude"].isna().sum())
print("Số dòng còn missing longitude:", df_cleaned["longitude"].isna().sum())

Số dòng còn missing latitude: 0
Số dòng còn missing longitude: 0


Có thể thấy các cột `latitude` và `longtitude` đã được điền khuyết đủ

### 3. Xử lý trùng lặp (De-duplication)
Trong dữ liệu quan trắc chất lượng không khí theo giờ, khóa định danh duy nhất của một dòng là: `(siteid, date)`

Vì:
- Mỗi trạm (siteid) phát 1 bản ghi tại mỗi thời điểm (date)
- Không thể có 2 bản ghi cùng thời gian tại cùng trạm

**Chiến lược:** Nếu tìm thấy trùng lặp, giữ lại bản ghi có **ít giá trị thiếu nhất** để tối đa hóa lượng thông tin giữ lại.


In [22]:
print("Số dòng bị trùng lặp: ",df_cleaned.duplicated(subset=["siteid", "date"]).sum())

Số dòng bị trùng lặp:  349333


#### Quan sát các giá trị bị trùng lặp

In [23]:
dup = df_cleaned[df_cleaned.duplicated(subset=["siteid", "date"], keep=False)]
dup.sort_values(["siteid", "date"])

,date,sitename,county,aqi,status,so2,co,o3,o3_8hr,pm10,pm2.5,no2,nox,no,windspeed,winddirec,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid
3125531,2020-11-26 03:00:00,Keelung,Keelung City,24.0,Good,NaN,0.20,27.5,25.00,21.0,10.0,6.8,7.7,1.0,NaN,NaN,0.3,7.0,19.0,2.0,121.760056,25.129167,1.0
3125616,2020-11-26 03:00:00,Keelung,Keelung City,24.0,Good,1.3,0.20,27.5,25.00,21.0,10.0,6.8,7.7,1.0,0.4,323.0,0.3,7.0,19.0,2.0,121.760056,25.129167,1.0
3125359,2020-11-26 04:00:00,Keelung,Keelung City,25.0,Good,1.3,0.19,28.4,27.00,15.0,7.0,7.3,8.1,0.8,2.0,83.0,0.3,7.0,18.0,2.0,121.760056,25.129167,1.0
3125397,2020-11-26 04:00:00,Keelung,Keelung City,25.0,Good,NaN,0.19,28.4,27.00,15.0,7.0,7.3,8.1,0.8,NaN,NaN,0.3,7.0,18.0,2.0,121.760056,25.129167,1.0
3113810,2020-12-01 19:00:00,Keelung,Keelung City,NaN,NaN,NaN,0.21,41.5,NaN,33.0,8.0,4.4,5.6,1.2,NaN,NaN,NaN,NaN,NaN,NaN,121.760056,25.129167,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855317,2022-03-03 11:00:00,Pingtung (Fangliao),Pingtung County,63.0,Moderate,2.0,0.78,45.6,17.00,61.0,30.0,NaN,NaN,NaN,0.9,264.0,0.4,20.0,39.0,0.0,120.590369,22.384742,313.0
1855016,2022-03-03 12:00:00,Pingtung (Fangliao),Pingtung County,70.0,Moderate,1.9,0.75,51.7,23.00,70.0,32.0,7.5,7.9,0.4,0.4,265.0,0.5,23.0,46.0,0.0,120.590369,22.384742,313.0
1855101,2022-03-03 12:00:00,Pingtung (Fangliao),Pingtung County,70.0,Moderate,1.9,0.75,51.7,23.00,70.0,32.0,7.5,7.9,0.4,0.4,265.0,0.5,23.0,46.0,0.0,120.590369,22.384742,313.0
496155,2023-11-13 10:00:00,Pingtung (Fangshan),Pingtung County,41.0,Good,1.1,0.29,39.8,36.43,49.0,27.0,4.9,5.6,0.7,1.7,282.0,0.2,12.7,28.0,0.0,120.651472,22.260899,313.0


#### Giữ lại bản ghi có **ít giá trị thiếu nhất**, xóa các bản ghi trùng lặp 

In [24]:
# 1. Đếm NaN trên mỗi dòng
df_cleaned["nan_count"] = df_cleaned.isna().sum(axis=1)

# 2. Sắp xếp theo siteid, date, nan_count (ít NaN đứng trước)
df_cleaned.sort_values(["siteid", "date", "nan_count"], inplace=True)

# 3. Loại duplicate theo siteid + date, giữ dòng ít NaN nhất
df_cleaned.drop_duplicates(subset=["siteid", "date"], keep="first", inplace=True)

# 4. Xóa cột phụ
df_cleaned.drop(columns=["nan_count"], inplace=True)

# 5. Reset index (không tạo copy lớn)
df_cleaned.reset_index(drop=True, inplace=True)

In [25]:
# Kiểm tra lại
print("Số dòng bị trùng lặp: ",df_cleaned.duplicated(subset=["siteid", "date"]).sum())

Số dòng bị trùng lặp:  0


#### Có thể thấy không còn dòng nào bị trùng lặp. Ta tiếp tục xử lý `missing value`

### 4. Điền khuyết dữ liệu đo lường (Imputation)

Dữ liệu chất lượng không khí là dữ liệu chuỗi thời gian (Time-series) có tính biến thiên liên tục. Việc điền bằng giá trị trung bình/trung vị đơn thuần có thể làm mất đi xu hướng tăng/giảm tự nhiên của nồng độ chất ô nhiễm.

Do đó, chúng ta áp dụng chiến lược **Time-series Interpolation** với 2 lớp xử lý:

1.  **Linear Interpolation (Ưu tiên 1 - Nội suy tuyến tính theo trạm):**
    *   Thực hiện nội suy tuyến tính trên từng nhóm trạm (`siteid`) đã được sắp xếp theo thời gian (`date`).
    *   Phương pháp này sẽ "vẽ" một đường thẳng nối giữa các điểm dữ liệu liền kề để điền vào khoảng trống, giúp bảo toàn xu hướng biến động của môi trường.
    *   Sử dụng `limit_direction='both'` để điền cả các giá trị thiếu ở đầu và cuối chuỗi.

2.  **Global Median (Ưu tiên 2 - Lưới an toàn):**
    *   Trong trường hợp hãn hữu một trạm bị thiếu **100% dữ liệu** của một chất (không có điểm mốc để nội suy), ta sẽ điền bằng giá trị trung vị (`median`) của toàn bộ tập dữ liệu để đảm bảo không còn sót lại giá trị `NaN` nào.

#### a. Các cột `pollutant`

In [26]:
# --- : Chuyển đổi dữ liệu sang số ---
pollutant_cols = [
    "so2", "co", "o3", "o3_8hr", "pm10", "pm2.5",
    "no2", "nox", "no", "co_8hr", "pm2.5_avg", "pm10_avg", "so2_avg"
]

# Đảm bảo các giá trị "-" hoặc bất kỳ giá trị phi số nào đều thành NaN
df_cleaned[pollutant_cols] = (
    df_cleaned[pollutant_cols]
    .apply(pd.to_numeric, errors="coerce")
)

# ---  Áp dụng nội suy tuyến tính ---

# 1.  Sắp xếp dữ liệu theo Trạm và Thời gian trước khi nội suy
# Nếu không sắp xếp, nội suy sẽ bị sai thứ tự thời gian
df_cleaned = df_cleaned.sort_values(by=["siteid", "date"])

# 2. Thực hiện nội suy theo từng trạm
for col in pollutant_cols:
    # Tính median toàn cục để dự phòng (như code cũ)
    median_all = df_cleaned[col].median()
    
    # Logic mới:
    # - interpolate(method='linear'): Nội suy tuyến tính (vẽ đường thẳng nối 2 điểm).
    # - limit_direction='both': Tự động điền cả các giá trị thiếu ở đầu và cuối chuỗi 
    #   (tương đương với việc ffill và bfill cho các điểm biên).
    # - fillna(median_all): Lưới an toàn cuối cùng. Nếu một trạm mất 100% dữ liệu (không nội suy được),
    #   nó sẽ lấy median toàn cục điền vào để không còn NaN.
    
    df_cleaned[col] = df_cleaned.groupby("siteid")[col].transform(
        lambda x: x.interpolate(method='linear', limit_direction='both').fillna(median_all)
    )

# Reset index sau khi sort để index gọn gàng
df_cleaned = df_cleaned.reset_index(drop=True)

In [27]:
# Danh sách tất cả các cột đã thực hiện điền khuyết
# (Bao gồm cả pollutant và các biến khí tượng/AQI)
cols_to_check = [
    "so2", "co", "o3", "o3_8hr", "pm10", "pm2.5",
    "no2", "nox", "no", "co_8hr", "pm2.5_avg", "pm10_avg", "so2_avg",
    "windspeed", "winddirec", "aqi"
]

print("--- Kiểm tra Missing Value sau khi điền khuyết ---")

# Đếm số lượng NaN trong các cột này
missing_counts = df_cleaned[cols_to_check].isna().sum()

# Lọc ra những cột vẫn còn missing (nếu có)
remaining_missing = missing_counts[missing_counts > 0]

if len(remaining_missing) == 0:
    print("Tuyệt vời! Tất cả các cột đã được điền đầy đủ (0 missing values).")
else:
    print("Cảnh báo: Vẫn còn giá trị thiếu ở các cột sau:")
    print(remaining_missing)
    
    # Hiển thị tỷ lệ % thiếu để đánh giá mức độ nghiêm trọng
    print("\nTỷ lệ % thiếu còn lại:")
    print((remaining_missing / len(df_cleaned) * 100).round(4).astype(str) + " %")

--- Kiểm tra Missing Value sau khi điền khuyết ---
Cảnh báo: Vẫn còn giá trị thiếu ở các cột sau:
windspeed    285689
winddirec    286516
aqi           48833
dtype: int64

Tỷ lệ % thiếu còn lại:
windspeed    5.1653 %
winddirec    5.1803 %
aqi          0.8829 %
dtype: object


#### b. Các cột `winddirec`, `windspeed`, `aqi`

In [28]:
# Danh sách các cột cần xử lý
numeric_cols = ["winddirec", "windspeed", "aqi"]

# 1. Đảm bảo các giá trị "-" hoặc bất kỳ giá trị phi số nào đều thành NaN
df_cleaned[numeric_cols] = (
    df_cleaned[numeric_cols]
    .apply(pd.to_numeric, errors="coerce")
)

# 2. Sắp xếp dữ liệu theo Trạm và Thời gian để đảm bảo nội suy đúng dòng thời gian
df_cleaned = df_cleaned.sort_values(by=["siteid", "date"])

# 3. Thực hiện nội suy tuyến tính (Linear Interpolation)
for col in numeric_cols:
    # Tính median toàn cục để dự phòng (trường hợp trạm đó mất 100% dữ liệu)
    median_all = df_cleaned[col].median()
    
    # - interpolate(method='linear'): Nội suy theo đường thẳng nối 2 điểm có dữ liệu
    # - limit_direction='both': Điền cả các giá trị thiếu ở đầu và cuối chuỗi
    # - fillna(median_all): Lưới an toàn cuối cùng nếu nội suy thất bại
    df_cleaned[col] = df_cleaned.groupby("siteid")[col].transform(
        lambda x: x.interpolate(method='linear', limit_direction='both').fillna(median_all)
    )

# Reset index sau khi sort để index gọn gàng
df_cleaned = df_cleaned.reset_index(drop=True)

# Kiểm tra nhanh kết quả
print("Đã hoàn tất nội suy cho:", numeric_cols)
print("Số lượng missing còn lại:\n", df_cleaned[numeric_cols].isna().sum())

Đã hoàn tất nội suy cho: ['winddirec', 'windspeed', 'aqi']
Số lượng missing còn lại:
 winddirec    0
windspeed    0
aqi          0
dtype: int64


### c. Cột `status`

- `status` là cột phân loại mức độ ô nhiễm dựa trên giá trị `AQI` (Good, Moderate, Unhealthy…).  
- Để đảm bảo **đồng nhất**, ta cập nhật lại `status` từ `AQI` đã điền khuyết theo chuẩn:  

| AQI | Status |
|-----|--------|
| 0–50 | Good |
| 51–100 | Moderate |
| 101–150 | Unhealthy for sensitive groups |
| 151–200 | Unhealthy |
| 201–300 | Very Unhealthy |
| >300 | Hazardous |


In [29]:
# Chỉ cập nhật các dòng mà AQI có và status khác với quy chuẩn
mask = df_cleaned["aqi"].notna() & (
    df_cleaned["status"] != df_cleaned["aqi"].apply(aqi_to_status)
)
df_cleaned.loc[mask, "status"] = df_cleaned.loc[mask, "aqi"].apply(aqi_to_status)

In [30]:
missing_info = check_missing(df_cleaned)
display(missing_info)

Total rows: 5,530,918
Columns with missing values: 0 / 23



,missing_count,missing_percent (%)
date,0,0.0
nox,0,0.0
latitude,0,0.0
longitude,0,0.0
so2_avg,0,0.0
pm10_avg,0,0.0
pm2.5_avg,0,0.0
co_8hr,0,0.0
winddirec,0,0.0
windspeed,0,0.0


#### Nhận xét:

- Đã xử lý toàn bộ `missing value`
- Kích thước dữ liệu còn lại: (5530946, 23)

## III. Xử lý outlier

Chúng ta sử dụng phương pháp **IQR (Interquartile Range)** để phát hiện các điểm dữ liệu bất thường.
* **Ngưỡng dưới:** $Q1 - 1.5 \times IQR$
* **Ngưỡng trên:** $Q3 + 1.5 \times IQR$

In [31]:
numeric_cols = [
    "aqi", "so2", "co", "o3", "o3_8hr", "pm10", "pm2.5",
    "no2", "nox", "no", "windspeed", "winddirec",
    "co_8hr", "pm2.5_avg", "pm10_avg", "so2_avg"
]


rows = []
for col in numeric_cols:
    s = df_cleaned[col]
    cnt, pct, lower, upper = count_outliers_iqr(s)
    rows.append({
        "column": col,
        "outlier_count": int(cnt),
        "outlier_percent": round(pct, 2),
        "lower_bound": round(lower, 2),
        "upper_bound": round(upper, 2),
    })

outlier_summary = pd.DataFrame(rows).sort_values("outlier_count", ascending=False)
outlier_summary

,column,outlier_count,outlier_percent,lower_bound,upper_bound
9,no,614737,11.11,-2.30,5.70
8,nox,348914,6.31,-11.50,35.70
2,co,268848,4.86,-0.14,0.74
1,so2,265841,4.81,-1.25,4.75
10,windspeed,255157,4.61,-1.60,5.60
7,no2,222530,4.02,-10.00,30.00
12,co_8hr,222198,4.02,-0.10,0.70
6,pm2.5,217630,3.93,-13.00,43.00
5,pm10,204514,3.70,-22.50,85.50
0,aqi,176228,3.19,-23.50,124.50


### 1. Kết quả Phân tích
Dựa trên bảng thống kê outlier phía trên, ta có các nhận định quan trọng:

* **Tỷ lệ Outlier cao ở nhóm khí thải:** Cột **`no` (Nitric Oxide)** có tỷ lệ outlier cao nhất (~11.6%), theo sau là **`nox`** và **`so2_avg`**. Điều này phản ánh đặc thù của khí thải giao thông (thường tăng đột biến vào giờ cao điểm).
* **Bản chất dữ liệu:** Trong bài toán môi trường, các giá trị cao đột biến thường là **Tín hiệu thật (Signal)** biểu thị các đợt ô nhiễm nghiêm trọng, không phải là Lỗi (Noise).

### 2. Quyết định Xử lý
Vì mục tiêu là phản ánh đúng hiện trạng môi trường, chúng ta áp dụng chiến lược sau:

1.  **Không xóa:** Việc xóa các dòng này sẽ làm mất đi thông tin về các sự kiện ô nhiễm quan trọng nhất.
2.  **Kiến nghị cho Mô hình hóa:**
    * Nếu sử dụng các mô hình nhạy cảm với outlier (như Hồi quy tuyến tính, kNN), cần áp dụng **Log Transformation** (Biến đổi Logarit) hoặc **Winsorization** (Giới hạn trần/sàn) để giảm độ lệch của phân phối.
    * Nếu sử dụng mô hình cây (Random Forest, XGBoost), có thể giữ nguyên dữ liệu này.

## IV Lưu dữ liệu sạch vào file csv

In [32]:
OUTPUT_PATH = "../data/processed/air_quality_processed.csv"
df_cleaned.to_csv(OUTPUT_PATH, index=False)
print(f"Dữ liệu sạch đã được lưu tại: {OUTPUT_PATH}")
print("Shape: ", df_cleaned.shape)

Dữ liệu sạch đã được lưu tại: ../data/processed/air_quality_processed.csv
Shape:  (5530918, 23)
